In [ ]:
from exp_train_network_energy_estimate_H2 import *
from multiprocessing import Pool
import multiprocessing  as mp

import scipy.io as scio

In [ ]:
eps = 0.15 ;  iters = 60;

In [ ]:
# generate 30 initial paramaters to train DQNN_1

counts_X = 30 ;    X_list = [4 * pi * np.random.random((2,8)) for i in range(counts_X)] ;

In [ ]:
# vary the residual ZZ interaction strengths
# choose 8 different interaction strengths from 0~4(MHz)

counts_H0 = 8 ;    f_list = np.arange(0, 4, 0.5) ;
H0_list = [generate_H0( 2 * pi * 0.001 * f_list[i] + np.zeros([7,7]) ) for i in range(counts_H0)] ;

In [ ]:
# vary the coherence time T in the simulation
# adjust T with different ratio (T/T0) to the coherence time T0 in the experiment
#  choose 5 different ratio (T/T0) : 10^(-1), 10^(-0.6), 10^(-0.2), 10^(0.2), 10^(0.6)

counts_time = 5;    time_list = array([-1 , -0.6, -0.2, 0.2, 0.6]) ;
c_ops = [decohere(time_list[i]) for i in range(counts_time) ] ;

In [ ]:
parameters = [ [ H0_list[i], c_ops[j], X_list[k] ]  
               for i in range(counts_H0) 
               for j in range(counts_time)
               for k in range(counts_X)    ] ;

In [ ]:
def train(para):
    _ , cost_history = train_network(para[2], eps, iters, para[0], para[1]) ;
    return cost_history

In [ ]:
if __name__ == "__main__":
    pool = Pool(processes = mp.cpu_count())
    energy_history = pool.map(train, parameters)
    pool.close()
    pool.join()

In [ ]:
# We observe that the DQNN will converge to the local minimum for 4 initial paramaters.
# To numerically investigate the effects of experimental errors in training the DQNN, we exclude 
# these 4 initial paramaters, and obtain the mean energy estimates 
# for other initial paramaters under different experimental errors. 

def average_remove_local_min(energy_estimates):
    return mean(sorted(energy_estimates)[0:25])

In [ ]:
energy_average = [average_remove_local_min([energy_history[i+j*counts_X][-1] for i in range(counts_X)]) 
               for j in range(counts_H0*counts_time)]
energy_average = np.reshape(energy_average, (counts_H0, counts_time)).T

In [ ]:
# scio.savemat('energy_history.mat', {'energy_history': energy_history, 'energy_average':energy_average})